# 第11章 接口：从协议到抽象基类

## 11.1 Python文化中的接口协议
引入抽象基类之前, Python就已经非常成功了, 即便是现在也很少有代码使用抽象基类。第一章中讨论了鸭子类型和协议。

在10.3节中，我们把协议定义为非正式的接口，是让Python这种动态类型语言实现多态的方式。

接口就是对象公开方法的子集，让对象在系统中扮演特定的角色。

## 11.2 Python喜欢序列

In [1]:
# 现在看一个Foo类
class Foo(object):
    
    def __getitem__(self, index):
        return range(0, 30, 10)[index]

In [2]:
f = Foo()

In [3]:
# 尽管没有实现iter方法, 但是f对象仍然是可迭代的
for i in f:
    print(i)

0
10
20


In [4]:
# 下面这个类是原来实现的FrenchDeck类
import collections
Card = collections.namedtuple("Card", ["rank", "suit"])

class FrenchDeck(object):
    ranks = [str(n) for n in range(2, 11)] + list("JQKA")
    suits = "spades diamonds clubs hearts".split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

## 11.3 使用猴子补丁在运行时实现协议

In [5]:
from random import shuffle

In [7]:
# 由于想要就地洗牌需要实现setitem方法, 所以必改进这个类, 但是为了重新写代码, 利用Python解释性语言的机制, 实现打补丁操作
deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [8]:
def set_card(deck, position, card):
    deck._cards[position] = card

In [9]:
FrenchDeck.__setitem__ = set_card

In [10]:
shuffle(deck)

In [11]:
deck[:5]

[Card(rank='7', suit='diamonds'),
 Card(rank='5', suit='clubs'),
 Card(rank='8', suit='spades'),
 Card(rank='K', suit='clubs'),
 Card(rank='7', suit='hearts')]

## 11.4 Alex Martelli的水禽 

In [14]:
class Struggle(object):
    def __len__(self):
        return 23

from collections import abc
# 这里并没有继承关系, 仅仅是自己写了一个len方法, 无需注册也可以将Struggle变为Sized的子类
isinstance(Struggle(), abc.Sized)

True

## 11.5 定义抽象基类的子类
我们建议先利用现有的抽象基类, 然后再斗胆定义自己的抽象基类

In [16]:
import collections

Card = collections.namedtuple("Card", ["rank", "suit"])

class FrenchDeck2(collections.abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list("JQKA")
    suits = "spades diamonds clubs hearts".split()
    
    def __init__(self):
        # 初始化牌类
        self._cards = [Card(rank, suit) for suit in suits
                                        for rank in ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
        
    def __delitem__(self, position):
        del self._cards[position]
        
    def insert(self, position, value):  # 尽管不需要这个方法, 但是需要为抽象基类提供实现
        self._cards.insert(position, value)

Python继承抽象类并不会在定义的时候检查, 而是在实例化的时候检查

## 11.6 标准库中的抽象类
从Python2.6开始, 标准库提供了抽象基类。大多数抽象基类在collections.abc模块中定义, 不过其他地方也有。例如numbers和io包中有一些抽象基类。

但是，collections.abc中的抽象基类最常用。

### 11.6.1 collections.abc模块中的抽象基类
Python里面有一个abc模块和collections.abc这两个有关抽象类的模块。

这部分需要结合着Python官方文档的UML图来看。

### 11.6.2 抽象基类的数字塔
numbers包定义的是“数字塔”，其中Number是位于最顶端的超类，随后是Complex子类，依次往下，最低端是Integer

## 11.7 定义并使用一个抽象基类

In [17]:
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """从可迭代对象中添加元素"""
        
    @abc.abstractclassmethod
    def pick(self):
        """随机删除元素，然后将其返回，如果实例为空，这个方法应该抛出LookupError"""
        
    def loaded(self):
        """如果至少一个元素，返回True，否则返回False"""
        return bool(self.inspect())
    
    def inspect(self):
        items = []
        while True:  # 这里用这个异常控制是因为并不知道子类会怎样储存
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)  # 把所有的item又load回去
        return tuple(sorted(items))

### 11.7.1 抽象基类句法详解
申明抽象基类最简单的方式是继承abc.ABC或者其他抽象基类

@abstractmethod和def语句之间不能有其他装饰器，也就是它必须放在装饰器最下层

### 11.7.2 定义Tombola抽象基类的子类
定义好Tombola抽象基类之后，我们还要开发两个具体子类，满足Tombola规定的接口。

In [18]:
import random

class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError("pick from empty BingoCage")
    
    def __call__(self):
        self.pick()

In [19]:
import random

class LotteryBlower(Tombola):
    
    def __init__(self, iterable):
        self._balls = list(iterable)
        
    def load(self, iterable):
        self._balls.extend(iterable)
    
    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError("pick from empty LotteryBlower")
        return self._balls.pop(position)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls))

### 11.7.3 Tombola的虚拟子类
白鹅类型的一个基本特性：即便不继承，也有办法把一个类注册为抽象基类的虚拟子类。这样做时，我们保证注册的类忠实得实现了抽象基类定义的接口，

而Python也会相信我们，从而不做检查。

In [20]:
from random import randrange

@Tombola.register
class TomboList(list):
    
    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError("pop from empty TomboList")
    
    load = list.extend  # 把list.extend方法给TomboList
    
    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))
    
    # Tombola.register(TomboList) Python3.3之前的版本不能把register函数作为类的装饰器

In [21]:
issubclass(TomboList, Tombola)

True

In [22]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

In [23]:
TomboList.__mro__  # TomboList其实并没有继承Tombola的任何属性和方法

(__main__.TomboList, list, object)